In [3]:
import cv2
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import time
import re

# ==========================================
# ⚙️ 設定區域
# ==========================================
# 影片對應表
DATASET_MAP = {
    ord('1'): "20230417_06_00230",
    ord('2'): "DSCN5480",
    ord('3'): "IMG_0004",
    ord('4'): "RCNX0029",
    ord('5'): "RCNX0062",
    ord('6'): "RCNX0279",
    ord('7'): "group3_20240429_0248",
    ord('8'): "group5_20240429_0289",
    ord('9'): "EK000061-20170904-1740",
    ord('0'): "IMG_0057"
}

GRID_SIZE = (3, 3)     
VIEW_WIDTH = 480       
VIEW_HEIGHT = 270      
HEADER_HEIGHT = 30     # 🔥 新增：標題列高度 (不會蓋住畫面)
FPS = 15               

# 九宮格規則
GRID_SLOTS = [
    ("1. Baseline + GT", ["predictions"], []),
    ("2. Majority (w=3)", ["majority", "w3"], []),
    ("3. Majority (w=5)", ["majority", "w5"], []),
    ("4. Majority (w=10)", ["majority", "w10"], []),
    ("5. Majority (All)", ["majority", "wall"], []),
    ("6. Weighted (w=3)", ["weighted", "w3"], []),
    ("7. Weighted (w=5)", ["weighted", "w5"], []),
    ("8. Weighted (w=10)", ["weighted", "w10"], []),
    ("9. Weighted (All)", ["weighted", "wall"], []),
]

# ==========================================
# 🛠️ 資源載入模組 (保持不變)
# ==========================================

def find_images(dataset_name):
    current_dir = Path.cwd()
    possible_paths = [
        current_dir / "data" / dataset_name / "images",
        current_dir.parent / "data" / dataset_name / "images",
        current_dir / dataset_name / "images",
        current_dir / "images"
    ]
    img_files = []
    for p in possible_paths:
        if p.exists():
            files = list(p.glob("*.jpg")) + list(p.glob("*.png"))
            if files: img_files = files; break
    
    if not img_files:
        all_imgs = list(current_dir.rglob("*.jpg")) + list(current_dir.rglob("*.png"))
        img_files = [f for f in all_imgs if dataset_name in str(f)]

    if img_files:
        img_files.sort(key=lambda x: int(''.join(filter(str.isdigit, x.name))) if any(c.isdigit() for c in x.name) else x.name)
        print(f"   🖼️  圖片: {len(img_files)} 張")
    else: print(f"   ❌ 找不到圖片")
    return img_files

def load_ground_truth(dataset_name, img_w, img_h):
    current_dir = Path.cwd()
    possible_paths = [
        current_dir / "data" / dataset_name / "labels",
        current_dir.parent / "data" / dataset_name / "labels",
        current_dir / dataset_name / "labels",
    ]
    labels_dir = None
    for p in possible_paths:
        if p.exists(): labels_dir = p; break
    if not labels_dir: return {}

    gt_dict = {}
    for f in labels_dir.glob("*.txt"):
        try:
            nums = re.findall(r'\d+', f.stem)
            if not nums: continue
            frame_id = int(nums[-1])
            with open(f, 'r') as file: lines = file.readlines()
            rows = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) < 5: continue
                cls_id = int(parts[0])
                xc, yc, w, h = map(float, parts[1:5])
                x1 = (xc - w/2) * img_w; y1 = (yc - h/2) * img_h
                x2 = (xc + w/2) * img_w; y2 = (yc + h/2) * img_h
                rows.append([x1, y1, x2, y2, -99, cls_id])
            gt_dict[frame_id] = rows
        except: pass
    print(f"   ✅ GT 載入成功 (共 {len(gt_dict)} 幀)")
    return gt_dict

def find_and_match_csvs(dataset_name):
    current_dir = Path.cwd()
    all_csvs = []
    for f in current_dir.rglob("*.csv"):
        if dataset_name in f.name: all_csvs.append(f)
    for f in current_dir.parent.rglob("*.csv"):
        if dataset_name in f.name: all_csvs.append(f)
    all_csvs = list(set(all_csvs))

    slot_files = [None] * 9
    for i, (title, keywords, _) in enumerate(GRID_SLOTS):
        for csv_file in all_csvs:
            fname = csv_file.name.lower()
            if all(k.lower() in fname for k in keywords):
                if "baseline" in title.lower():
                    if "voted" not in fname and "majority" not in fname:
                         slot_files[i] = csv_file; break
                else:
                    if "predictions" not in fname:
                        slot_files[i] = csv_file; break
    return slot_files

def load_csv_data(csv_path):
    if not csv_path: return None, set()
    try:
        df = pd.read_csv(csv_path)
        df.columns = df.columns.str.strip()
        if all(c in df.columns for c in ['x', 'y', 'w', 'h']):
            df['x1'] = df['x']; df['y1'] = df['y']
            df['x2'] = df['x'] + df['w']; df['y2'] = df['y'] + df['h']
        cls_col = 'cls_id' if 'cls_id' in df.columns else 'class_id'
        if cls_col not in df.columns: df[cls_col] = 0
        if 'obj_id' not in df.columns: df['obj_id'] = -1 
        data_dict = {}
        df['frame_id'] = df['frame_id'].astype(int)
        for frame_id, group in df.groupby('frame_id'):
            rows = group[['x1', 'y1', 'x2', 'y2', 'obj_id', cls_col]].values.tolist()
            data_dict[frame_id] = rows
        return data_dict, set(df['frame_id'].unique())
    except: return None, set()

# ==========================================
# 🛠️ 核心繪圖邏輯 (修正版：獨立標題區)
# ==========================================
def draw_cell(img, title, boxes, gt_boxes=None, show_gt=True):
    # 建立一個新的大畫布：高度 = 圖片高 + 標題高
    h, w = img.shape[:2]
    total_h = h + HEADER_HEIGHT
    canvas_cell = np.zeros((total_h, w, 3), dtype=np.uint8)
    
    # 1. 填入標題 (在最上方)
    cv2.rectangle(canvas_cell, (0, 0), (w, HEADER_HEIGHT), (20, 20, 20), -1) # 深灰色背景
    
    # 標題文字
    info_text = title
    if show_gt and gt_boxes: info_text += " (Grn=GT)"
    cv2.putText(canvas_cell, info_text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # 2. 貼上圖片 (在標題下方)
    canvas_cell[HEADER_HEIGHT:, :] = img
    
    # --- 定義座標位移量 (所有框框都要往下移) ---
    y_offset = HEADER_HEIGHT

    # 內部函式：智慧標籤 (含位移與邊界檢查)
    def draw_smart_label(bbox, text, color, is_gt=False):
        # 取出原始座標，並加上位移
        x1, y1, x2, y2 = bbox
        y1 += y_offset
        y2 += y_offset
        
        # 文字大小
        font_scale = 0.5; thickness = 1
        (t_w, t_h), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
        
        # 決定 Y 座標 (防裁切策略)
        # 邊界現在是 HEADER_HEIGHT (上方) 和 total_h (下方)
        
        if is_gt: # GT 優先放下面
            if y2 + t_h + 5 > total_h:
                text_y = y2 - 5         # 移入框內底部
            else:
                text_y = y2 + t_h + 5   # 框外下方
        else:     # 預測優先放上面
            # 如果上方空間會撞到標題列 (y < HEADER_HEIGHT)
            if y1 - t_h - 5 < HEADER_HEIGHT:
                text_y = y1 + t_h + 10  # 移入框內頂部
            else:
                text_y = y1 - 5         # 框外上方

        # 決定 X 座標
        if x1 + t_w > w: text_x = w - t_w - 2
        else: text_x = max(0, x1)

        # 繪圖
        cv2.rectangle(canvas_cell, (text_x, text_y - t_h - 2), (text_x + t_w, text_y + baseline), color, -1)
        cv2.putText(canvas_cell, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness)

    # 3. 繪製 GT (加上位移)
    if show_gt and gt_boxes:
        for box in gt_boxes:
            x1, y1, x2, y2, _, cls_id = map(int, box)
            # 畫框 (記得加位移)
            cv2.rectangle(canvas_cell, (x1, y1+y_offset), (x2, y2+y_offset), (0, 255, 0), 1)
            draw_smart_label([x1, y1, x2, y2], f"GT:{cls_id}", (0, 255, 0), is_gt=True)

    # 4. 繪製預測框
    if boxes:
        for box in boxes:
            x1, y1, x2, y2, obj_id, cls_id = map(int, box)
            if obj_id == -1: color = (0, 0, 255); label = f"Det:{cls_id}"
            else: np.random.seed(int(obj_id)); color = np.random.randint(60, 255, 3).tolist(); label = f"{obj_id}:{cls_id}"
            
            cv2.rectangle(canvas_cell, (x1, y1+y_offset), (x2, y2+y_offset), color, 2)
            draw_smart_label([x1, y1, x2, y2], label, color, is_gt=False)

    return canvas_cell

def load_dataset_resources(dataset_name):
    print(f"\n🔄 切換至: {dataset_name}")
    img_files = find_images(dataset_name)
    if not img_files: return None, None, None, None
    
    sample_img = cv2.imread(str(img_files[0]))
    img_h, img_w = sample_img.shape[:2]
    gt_data = load_ground_truth(dataset_name, img_w, img_h)
    slot_files = find_and_match_csvs(dataset_name)
    exp_data_list = []
    all_active = set(gt_data.keys())
    
    print("   📊 載入數據...")
    for f in slot_files:
        data, valid_frames = load_csv_data(f)
        exp_data_list.append(data)
        all_active.update(valid_frames)
    return img_files, exp_data_list, gt_data, sorted(list(all_active))

# ==========================================
# 🚀 主程式
# ==========================================
def main():
    print(f"📍 程式啟動")
    print("⌨️  按鍵說明: [1-9,0] 切換影片 | [G] 開關 GT | [N] 跳轉 | [Space] 暫停")
    
    current_key = ord('1')
    current_dataset = DATASET_MAP[current_key]
    img_files, exp_data_list, gt_data, active_frames = load_dataset_resources(current_dataset)
    
    idx = 0; paused = True; show_gt = True
    
    # 計算主視窗大小 (包含 Header)
    CELL_H = VIEW_HEIGHT + HEADER_HEIGHT
    CELL_W = VIEW_WIDTH
    
    while True:
        if not img_files:
            canvas = np.zeros((CELL_H*3, CELL_W*3, 3), dtype=np.uint8)
            cv2.putText(canvas, "No Images. Press 1-9.", (50, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
            cv2.imshow("Player", canvas); key = cv2.waitKey(100) & 0xFF
        else:
            img_path = img_files[idx]
            try: frame_id = int(''.join(filter(str.isdigit, img_path.stem)))
            except: frame_id = idx + 1
            
            raw_img = cv2.imread(str(img_path))
            if raw_img is None: break
            orig_h, orig_w = raw_img.shape[:2]
            
            # 初始化大畫布
            canvas = np.zeros((CELL_H*3, CELL_W*3, 3), dtype=np.uint8)
            
            # GT
            current_gt_boxes = []
            if gt_data and frame_id in gt_data:
                sx = VIEW_WIDTH / orig_w; sy = VIEW_HEIGHT / orig_h
                current_gt_boxes = [[b[0]*sx, b[1]*sy, b[2]*sx, b[3]*sy, b[4], b[5]] for b in gt_data[frame_id]]

            for i in range(9):
                row = i // 3; col = i % 3
                title = GRID_SLOTS[i][0]
                view = cv2.resize(raw_img, (VIEW_WIDTH, VIEW_HEIGHT))
                data_dict = exp_data_list[i]
                
                boxes = []
                if data_dict and frame_id in data_dict:
                    sx = VIEW_WIDTH / orig_w; sy = VIEW_HEIGHT / orig_h
                    boxes = [[b[0]*sx, b[1]*sy, b[2]*sx, b[3]*sy, b[4], b[5]] for b in data_dict[frame_id]]
                
                # 繪製單格 (包含 Header)
                view_with_header = draw_cell(view, title, boxes, 
                               gt_boxes=current_gt_boxes if i == 0 else None, 
                               show_gt=show_gt)
                
                # 貼上大畫布 (注意座標計算要用 CELL_H)
                y_s = row * CELL_H; x_s = col * CELL_W
                canvas[y_s:y_s+CELL_H, x_s:x_s+CELL_W] = view_with_header
            
            info = f"[{current_dataset}] Frame: {frame_id} ({idx+1}/{len(img_files)})"
            if show_gt: info += " [GT: ON]"
            # 狀態列也往下移
            cv2.putText(canvas, info, (20, CELL_H*3 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            cv2.imshow("Player With Header", canvas)
            key = cv2.waitKey(0 if paused else int(1000/FPS)) & 0xFF

        if key == ord('q'): break
        elif key == ord(' '): paused = not paused
        elif key == ord('g'): show_gt = not show_gt
        elif key == ord('d'): idx = min(idx + 1, len(img_files)-1) if img_files else 0
        elif key == ord('a'): idx = max(idx - 1, 0) if img_files else 0
        elif key == ord('n'):
            if active_frames:
                targets = [f for f in active_frames if f > frame_id]
                if targets:
                    t = targets[0]
                    for k in range(idx+1, len(img_files)):
                        try:
                            if int(''.join(filter(str.isdigit, img_files[k].stem))) == t: idx = k; break
                        except: pass
        elif key in DATASET_MAP:
            if key != current_key:
                current_key = key
                current_dataset = DATASET_MAP[key]
                img_files, exp_data_list, gt_data, active_frames = load_dataset_resources(current_dataset)
                idx = 0; paused = True

        if not paused and img_files:
            idx += 1
            if idx >= len(img_files): idx = 0

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

📍 程式啟動
⌨️  按鍵說明: [1-9,0] 切換影片 | [G] 開關 GT | [N] 跳轉 | [Space] 暫停

🔄 切換至: 20230417_06_00230
   🖼️  圖片: 200 張
   ✅ GT 載入成功 (共 200 幀)
   📊 載入數據...

🔄 切換至: DSCN5480
   🖼️  圖片: 766 張
   ✅ GT 載入成功 (共 766 幀)
   📊 載入數據...
